In [4]:
import datetime
import myfitnesspal
import gspread 


start_date = datetime.date(datetime.date.today().year, 1, 1)  # Get the first day of the current year
end_date = datetime.date.today()  # Get today's date
mfp = myfitnesspal.Client()
sheets = gspread.service_account('./service_account.json')
sh = sheets.open('HealthTracking')
# lastUpdated = -1
print(sh.worksheets())

# export daily calories to google sheets


# day = mfp.get_date(2023, 4, 10)


for date in (start_date + datetime.timedelta(n) for n in range((end_date - start_date).days + 1)):
    try:
        day = mfp.get_date(date.year, date.month, date.day)
        worksheet = sh.worksheet("Sheet1")  # Replace "Sheet1" with the name of your worksheet
        worksheet.update_cell((date - start_date).days + 1, 1, date.strftime('%Y-%m-%d'))  # Update the first cell with the date
        worksheet.update_cell((date - start_date).days + 1, 2, day.totals['calories'])  # Update the second cell with the total calories
        worksheet.update_cell((date - start_date).days + 1, 3, day.totals['carbohydrates'])  # Update the second cell with the total calories
        worksheet.update_cell((date - start_date).days + 1, 4, day.totals['fat'])  # Update the second cell with the total calories
        worksheet.update_cell((date - start_date).days + 1, 5, day.totals['protein'])  # Update the second cell with the total calories
        print(f"Successfully updated worksheet for {date.strftime('%Y-%m-%d')}")
    except myfitnesspal.exceptions.MyfitnesspalAuthenticationError:
        print(f"No data found for {date.strftime('%Y-%m-%d')}")  # Update the second cell with the total calories

[<Worksheet 'Sheet1' id:0>]


KeyError: 'calories'